In [2]:
import preprocess as pp
import configs
import persister
import relation
import lda
import json

from stanfordcorenlp import StanfordCoreNLP
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.tree import Tree
from imp import reload

inited stanford CoreNLP client, dont forget to close it!


In [ ]:
# 文档向量化测试
# count vectorizer

In [ ]:
a = "Many existing knowledge bases(KBs), including Freebase, Yago, and NELL, rely on a ﬁxed ontology, given as an input to the system, which deﬁnes the data to be cataloged in the KB, i.e., a hierarchy of categories and relations between them. The system then extracts facts that match the predeﬁned ontology. We propose an unsupervised model that jointly learns a latent ontological structure of an input corpus, and identiﬁes facts from the corpus that match the learned structure. Our approach combines mixed membership stochastic block models and topic models to infer a structure by jointly modeling text, a latent concept hierarchy, and latent semantic relationships among the entities mentioned in the text. As a case study, we apply the model to a corpus of Web documents from the software domain,and evaluate the accuracy of the various components of the learned ontology. "
b = ["hello world. what the fuck."]

In [ ]:
c = [" ".join(preprocess(a))]
vector = CountVectorizer(ngram_range=(1, 2), vocabulary=["knowledge base"], stop_words='english')
vector.build_analyzer()
x = vector.fit_transform(c)

In [ ]:
estimator = CountVectorizer(ngram_range=(1, 2), vocabulary=['fuck', 'hello', 'world', "hello world"], stop_words='english')
estimator.build_analyzer()
res = estimator.fit_transform(b)

In [ ]:
estimator.get_feature_names()

In [ ]:
res.toarray()
# estimator.vocabulary_

In [ ]:
# 斯坦福nlp工具测试
# stanfordcorenlp

In [ ]:
nlp = StanfordCoreNLP('/mnt/d/stanford-corenlp-full-2018-02-27', port=9000)
sentence = 'Brink is taking part in the final exercise.'
print('Tokenize:', nlp.word_tokenize(sentence))
print('Part of Speech:', nlp.pos_tag(sentence))
print('Named Entities:', nlp.ner(sentence))
print('Constituency Parsing:', nlp.parse(sentence))
print('Dependency Parsing:', nlp.dependency_parse(sentence))

nlp.close() # Do not forget to close! The backend server will consume a lot memery.

In [ ]:
# 官方corenlp api
# stanza

In [ ]:
# 新闻数据集测试
# 20newsgroups

In [19]:
import newsgroups
size = 500
tmp = newsgroups.get_news_data(size)
newsdata = []
for cate in tmp:
    newsdata.extend(cate)

In [ ]:
texts = [' '.join(pp.preprocess_abstract(a)) for a in tmp]
persister.save_json(configs.NEWSDATA, texts)

In [ ]:
# 处理成句子
# nltk.sent_tokenize

In [ ]:
newssent = []
for news in newsdata:
    newssent.append(pp.split2sent(news))
persister.save_json(configs.NEWSSENT, newssent)

In [ ]:
newssenttoken = []
for news in newssent:
    tokenized_lemmatized_news = []
    for sent in news:
        tokenized_lemmatized_news.append(relation.lemmatize_sent_words(sent))
    newssenttoken.append(tokenized_lemmatized_news)
persister.save_json(configs.NEWSSENTTOKEN, newssenttoken)

In [ ]:
# load preprossed data

In [3]:
texts = persister.load_json(configs.NEWSDATA)
newssent = persister.load_json(configs.NEWSSENT)
newssenttoken = persister.load_json(configs.NEWSSENTTOKEN)
newsparse = []
with open(configs.NEWSPARSE+".json", encoding="utf8") as f:
    for l in f:
        newsparse.append(json.loads(l))
# newsparse = [json.loads(n) for n in newsparse]

In [ ]:
print(newssent[1][0])
email_reg = "[0-9a-zA-Z_]{0,19}@(?:[0-9a-zA-Z]{1,13}[.])+\w+"
reg = re.compile(email_reg)
g = reg.findall(newssent[1][0])
print(g)

In [ ]:
# load lda res
terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA)
lda.print_topics(topic_word, terms, doc_topic)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic)

In [4]:
idx_news = 1195
idx_sent = 40
print(len(newssent[idx_news]))
print(newssent[idx_news][idx_sent])
print(newssenttoken[idx_news][idx_sent])
print(texts[idx_news])

46
My father and I built a "Veep" (Volkswagen powered Jeep CJ-2A) when I was in high school, so I consider myself fairly good with tools, electronics, and cars.
['My', 'father', 'and', 'I', 'built', 'a', '"', 'Veep', '"', '(', 'Volkswagen', 'powered', 'Jeep', 'CJ-2A', ')', 'when', 'I', 'wa', 'in', 'high', 'school', ',', 'so', 'I', 'consider', 'myself', 'fairly', 'good', 'with', 'tool', ',', 'electronics', ',', 'and', 'car', '.']
howdy little new newsgroup would like tap knowledge expertise available subject market cruise control background recently broke ankle road bicycling accident place five screw yuk two week returning texas home school byu provo utah imagine trying drive nearly mile broken right ankle epitome good time car doe cruise control would pedalling ha ha messed ankle question general opinion market cruise control unit realize cheap cc cruise control say pep boy going good factory professionally installed unit thing uderstand probably expect much way accuracy look sort thi

In [ ]:
newsdata[1200]

In [10]:
newssent[1200]

['In <smN42B1w165w@cybernet.cse.fau.edu> vlasis@cybernet.cse.fau.edu (vlasis theodore) writes:  >tobias@convex.com (Allen Tobias) writes:  >> In article <1993Apr15.024246.8076@Virginia.EDU> ejv2j@Virginia.EDU ("Erik Vel >> >This happened about a year ago on the Washington DC Beltway.',
 '>> >Snot nosed drunken kids decided it would be really cool to >> >throw huge rocks down on cars from an overpass.',
 'Four or five >> >cars were hit.',
 'There were several serious injuries, and sadly >> >a small girl sitting in the front seat of one of them was struck  >> >in the head by one of the larger rocks.',
 "I don't recall if she  >> >made it, but I think she was comatose for a month or so and  >> >doctors weren't holding out hope that she'd live.",
 '>> > >> >What the hell is happening to this great country of ours?',
 'I >> >can see boyhood pranks of peeing off of bridges and such, but >> >20 pound rocks??!',
 'Has our society really stooped this low??',
 '>> > >> >Erik velapold >>  >> Soci

In [17]:
[" ".join([w["originalText"] for w in sent["tokens"]]) for sent in newsparse[1200]["sentences"]]

['In <smN42B1w165w@cybernet.cse.fau.edu> vlasis@cybernet.cse.fau.edu ( vlasis theodore ) writes : > tobias@convex.com ( Allen Tobias ) writes : >> In article <1993Apr15.024246.8076@Virginia.EDU> ejv2j@Virginia.EDU ( " Erik Vel >> > This happened about a year ago on the Washington DC Beltway .',
 '>> > Snot nosed drunken kids decided it would be really cool to >> > throw huge rocks down on cars from an overpass .',
 'Four or five >> > cars were hit .',
 'There were several serious injuries , and sadly >> > a small girl sitting in the front seat of one of them was struck >> > in the head by one of the larger rocks .',
 "I do n't recall if she >> > made it , but I think she was comatose for a month or so and >> > doctors were n't holding out hope that she 'd live .",
 '>> > >> > What the hell is happening to this great country of ours ?',
 'I >> > can see boyhood pranks of peeing off of bridges and such , but >> > 20 pound rocks ??!',
 'Has our society really stooped this low ??',
 '>> > 

In [ ]:
print(pos_tag(word_tokenize(newssent[idx_news][43])))
print(pp.WNL.lemmatize("has", wordnet.VERB))

In [ ]:
print(len(newsparse[idx_news]["sentences"]))
print(newsparse[idx_news]["sentences"][idx_sent].keys())
examparse = newsparse[idx_news]["sentences"][idx_sent]
print(examparse["enhancedPlusPlusDependencies"])

In [ ]:
tt = Tree.fromstring(examparse["parse"])
tt.pretty_print()

In [ ]:
t4 = [t[0] for t in top_terms[3]]
t4_values = [t[1] for t in top_terms[3]]
print(t4)
print(t4_values)
idxs, importance, count = relation.extract_important_sents(newssenttoken[idx_news], t4, t4_values)
print(idxs, importance[40], count[40])

In [ ]:
relation.extract_word_relation_from_sent(i, newsparse[1200]["sentences"][sent_idx]["enhancedDependencies"])

In [ ]:
for term in top_terms[3]:
    for news in top_docs[3]:
        time.sleep(2)
        news_idx = news[0]
        for sent_idx,sent in enumerate(newssenttoken[news_idx]):
            if type(newsparse[news_idx]) is str:
                print(news_idx, "parse", "err")
                break
            topic_word_idxs = relation.get_word_idx(term[0], sent)
            for i in topic_word_idxs:
                print(sent_idx, i, news_idx)
                rs = relation.extract_word_relation_from_sent(i, newsparse[news_idx]["sentences"][sent_idx]["enhancedDependencies"])
                for r in rs:
                    print(relation.convert_relation2str(r))

In [ ]:
t = Tree.fromstring(newsparse[idx_news]["sentences"][5]["parse"])
t.pretty_print()

In [ ]:
# test example res

In [ ]:
import json
with open("exampleres.json",encoding="utf8")as f:
    res = json.load(f)
print(res)